In [1]:
import numpy as np 
import pandas as pd 
from pathlib import Path  
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import re
import spacy
from spacy.lang.en import English 
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential



Data Loading:

In [2]:
file_path = '/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv'
se = pd.read_csv(file_path)


Preprocessing:

In [3]:
se.dropna(subset=['wording', 'content'], inplace=True)
se['text'].fillna('', inplace=True)

# Preprocess text function
def preprocess_text(text):
    text = text.replace('\n', ' ').lower()  # replace newlines and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # remove punctuation
    text = ' '.join([word for word in text.split() if word.isalpha() and word not in stop_words])  # remove stop words
    return ' '.join(text.split())  # replace sequences of whitespace

stop_words = spacy.lang.en.stop_words.STOP_WORDS
preprocessed_texts = [preprocess_text(text) for text in se['text']]
se['PreprocessedText'] = preprocessed_texts


Tokenization and Vectorization using Word2Vec:

In [4]:
sentences = se['PreprocessedText'].apply(lambda x: x.split()).tolist()
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
vocabulary = set(w2v_model.wv.index_to_key)


Feature Creation

In [5]:
def average_word_vectors(words, w2v_model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    n_words = 0.
    for word in words:
        if word in vocabulary: 
            n_words = n_words + 1.
            feature_vector = np.add(feature_vector, w2v_model.wv[word])
    if n_words:
        feature_vector = np.divide(feature_vector, n_words)
    return feature_vector

features = [average_word_vectors(tokens, w2v_model, vocabulary, 100) for tokens in sentences]


Split Data

In [6]:
y = se[['wording', 'content']].values
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)




Model Creation and Training

In [7]:
# Scale the feature vectors
X_train_dense = np.array(X_train)
X_test_dense = np.array(X_test)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_dense)
X_test_scaled = scaler.transform(X_test_dense)

# Define and compile the model
model = Sequential()
model.add(Dense(116, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(2, activation='linear'))  # Two output nodes for "wording" and "content"
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
144/144 [==============================] - 1s 4ms/step - loss: 0.9528 - mse: 0.9528 - val_loss: 0.8456 - val_mse: 0.8456
Epoch 2/50
144/144 [==============================] - 0s 2ms/step - loss: 0.8452 - mse: 0.8452 - val_loss: 0.8109 - val_mse: 0.8109
Epoch 3/50
144/144 [==============================] - 0s 2ms/step - loss: 0.8117 - mse: 0.8117 - val_loss: 0.8264 - val_mse: 0.8264
Epoch 4/50
144/144 [==============================] - 0s 2ms/step - loss: 0.7884 - mse: 0.7884 - val_loss: 0.7787 - val_mse: 0.7787
Epoch 5/50
144/144 [==============================] - 0s 2ms/step - loss: 0.7805 - mse: 0.7805 - val_loss: 0.8332 - val_mse: 0.8332
Epoch 6/50
144/144 [==============================] - 0s 2ms/step - loss: 0.7530 - mse: 0.7530 - val_loss: 0.7503 - val_mse: 0.7503
Epoch 7/50
144/144 [==============================] - 0s 2ms/step - loss: 0.7488 - mse: 0.7488 - val_loss: 0.7475 - val_mse: 0.7475
Epoch 8/50
144/144 [==============================] - 0s 2ms/step - loss: 0.

Evaluation:

In [8]:
# Predict on test data
y_pred = model.predict(X_test_scaled)
rmse_wording = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)
rmse_content = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)

print(f"RMSE for wording: {rmse_wording}")
print(f"RMSE for content: {rmse_content}")


45/45 [==============================] - 0s 1ms/step
RMSE for wording: 0.8262320192421719
RMSE for content: 0.7818685111494186


Competition Test Set Prediction:

In [9]:
test_df = pd.read_csv(Path("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv"))

test_df['PreprocessedText'] = test_df['text'].apply(preprocess_text)
test_sentences = test_df['PreprocessedText'].apply(lambda x: x.split()).tolist()
X_test_competition = [average_word_vectors(tokens, w2v_model, vocabulary, 100) for tokens in test_sentences]
X_test_competition = np.array(X_test_competition)

# Predict using the trained model
predictions = model.predict(X_test_competition)



1/1 [==============================] - 0s 23ms/step


Submission Creation:

In [10]:
submission_df = pd.DataFrame({
    'student_id': test_df['student_id'],
    'content': predictions[:, 1],
    'wording': predictions[:, 0]
})
submission_df['content'] = np.clip(submission_df['content'], -2, 5)
submission_df['wording'] = np.clip(submission_df['wording'], -2, 5)
submission_df.to_csv("submission.csv", index=False)
